In [78]:
import numpy as np
import pandas as pd
import os
import re


In [79]:
category = {
"Balance_Sheet" : [
                "Total Share Capital",
                "Reserves and Surplus",
                "Total Reserves and Surplus",
                "Total Shareholders Funds",
                "Other Current Liabilities",
                "Total Current Liabilities",
                "Total Capital And Liabilities",
                "Tangible Assets",
                "Fixed Assets",
                "Total Non-Current Assets",
                "Total Current Assets",
                "Total Assets",
                ],

"Cash_Flow" : [
                "Net Profit/Loss Before Extraordinary Items And Tax",
                "Net CashFlow From Operating Activities",
                "Net Cash Used In Investing Activities",
                "Net Cash Used From Financing Activities",
                "Foreign Exchange Gains / Losses",
                "Net Inc/Dec In Cash And Cash Equivalents",
                "Cash And Cash Equivalents Begin of Year",
                "Cash And Cash Equivalents End Of Year",
                ],

"Profit_Loss" : [ 
                "Revenue From Operations [Gross]",
                "Revenue From Operations [Net]",
                "Total Operating Revenues",
                "Other Income",
                "Total Revenue",
                "Operating And Direct Expenses",
                "Employee Benefit Expenses",
                "Finance Costs",
                "Depreciation And Amortisation Expenses",
                "Other Expenses",
                "Total Expenses",
                "Profit/Loss Before Exceptional, ExtraOrdinary Items And Tax",
                "Profit/Loss Before Tax",
                "Current Tax",
                "Deferred Tax",
                "Total Tax Expenses",
                "Profit/Loss After Tax And Before ExtraOrdinary Items",
                "Profit/Loss From Continuing Operations",
                "Profit/Loss For The Period",
                "Basic EPS (Rs.)",
                "Diluted EPS (Rs.)",
               ],

"Quarterly" : [        
                "Net Sales/Income from operations",
                "Total Income From Operations",
                "Employees Cost",
                "depreciat",
                "Other Expenses",
                "P/L Before Other Inc. , Int., Excpt. Items & Tax",
                "Other Income",
                "P/L Before Int., Excpt. Items & Tax",
                "Interest",
                "P/L Before Exceptional Items & Tax",
                "P/L Before Tax",
                "Tax",
                "P/L After Tax from Ordinary Activities",
                "Net Profit/(Loss) For the Period",
                "Equity Share Capital",
                "Basic EPS",
                "Diluted EPS",
                "Basic EPS.",
                "Diluted EPS.",
        ],

"Ratio" : [        
                "Revenue from Operations/Share (Rs.)",
                "PBDIT/Share (Rs.)",
                "PBIT/Share (Rs.)",
                "PBT/Share (Rs.)",
                "Net Profit/Share (Rs.)",
                "PBDIT Margin (%)",
                "PBIT Margin (%)",
                "PBT Margin (%)",
                "Net Profit Margin (%)",
                "Return on Networth / Equity (%)",
                "Return on Assets (%)",
                "Total Debt/Equity (X)",
                "Dividend Payout Ratio (NP) (%)",
                "Dividend Payout Ratio (CP) (%)",
                "Earnings Retention Ratio (%)",
                "Enterprise Value (Cr.)",
                "EV/EBITDA (X)",
        ],
}

In [80]:
# Input and output paths
input_folder = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting\3i Infotech Ltd\Excel"
output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS_2.xlsx"
new_output_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS_3.xlsx"


In [81]:
# Mapping filenames to category keys
file_category_mapping = {
    "Balance-sheet_combined.xlsx": "Balance_Sheet",
    "Cash-flow_combined.xlsx": "Cash_Flow",
    "Profit-loss_combined.xlsx": "Profit_Loss",
    "Quarterly-resul_combined.xlsx": "Quarterly",
    "Ratios_combined.xlsx": "Ratio",
}

In [82]:

# Create an empty dictionary to store cleaned data for each sheet
cleaned_data = {}


In [83]:
def extract_year(column_name):
    match = re.search(r'\d+', str(column_name))  # Extract the first number found
    return int(match.group()) if match else float('inf')  # Assign high value if no number

In [84]:
# Process each file
for filename, cat_key in file_category_mapping.items():
    file_path = os.path.join(input_folder, filename)
    if not os.path.exists(file_path):
        print(f"Skipping {filename} (File not found)")
        continue
    
    # Read Excel file
    df = pd.read_excel(file_path)
    df_parameters = df.iloc[:, 0].astype(str).tolist()
    
    # Identify parameters not in category
    not_common_parameters = [item for item in df_parameters if item not in category[cat_key]]
    
    # Drop non-matching rows
    df = df[~df.iloc[:, 0].isin(not_common_parameters)]
    df.replace("12 mths", np.nan, inplace=True)
    df.dropna(inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    
    # Sort columns by year
    column_start = df.columns[0]
        # Sort columns by extracted year values
    sorted_columns = sorted(df.columns[1:], key=extract_year)

    # Reorder DataFrame columns
    df = df[[column_start] + sorted_columns]

    # # Display the first few rows
    # df.head(3)
    
    # Reset index and rename headers
    df = df.T.reset_index()
    
    df.to_excel(new_output_file,index=False)
    correct_headers = [column_start] + [
    item if item.lower() == "depreciat" else item.capitalize() 
    for item in category[cat_key]
    ]
    
    print(df.columns)
    print(len(df.columns))
    
    print(correct_headers)
    print(len(correct_headers))
    df.columns = correct_headers
    
    # Remove redundant first row if needed
    if (df.iloc[0] == df.columns).all():
        df = df[1:].reset_index(drop=True)
    
    # Store cleaned dataframe
    cleaned_data[cat_key] = df

Index(['index', 4, 5, 6, 7, 17, 19, 20, 23, 27, 32, 40, 41], dtype='object')
13
['Balance Sheet of 3i Infotech(in Rs. Cr.)', 'Total share capital', 'Reserves and surplus', 'Total reserves and surplus', 'Total shareholders funds', 'Other current liabilities', 'Total current liabilities', 'Total capital and liabilities', 'Tangible assets', 'Fixed assets', 'Total non-current assets', 'Total current assets', 'Total assets']
13
Index(['index', 1, 2, 3, 4, 5, 7, 8, 9], dtype='object')
9
['Cash Flow of 3i Infotech(in Rs. Cr.)', 'Net profit/loss before extraordinary items and tax', 'Net cashflow from operating activities', 'Net cash used in investing activities', 'Net cash used from financing activities', 'Foreign exchange gains / losses', 'Net inc/dec in cash and cash equivalents', 'Cash and cash equivalents begin of year', 'Cash and cash equivalents end of year']
9
Index(['index',       2,       4,       5,       6,       7,      11,      13,
            14,      15,      16,      17,      1

In [86]:
# Save all dataframes to a single Excel file with multiple sheets
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for sheet_name, df in cleaned_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Processing complete. Cleaned data saved to", output_file)

Processing complete. Cleaned data saved to C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Data Preporcessing\3i infotech\cleanedDataBS_2.xlsx
